In [1]:
import pandas as pd

NDX = "./ndx.csv"

In [2]:
ndx = pd.read_csv(NDX, index_col=0, parse_dates=True, usecols= [0,5], header=0, names=['date', 'close'])
ndx['dailyReturn'] = ndx['close'].pct_change()
ndx

,close,dailyReturn
date,,
1985-10-01,112.139999,NaN
1985-10-02,110.824997,-0.011726
1985-10-03,110.870003,0.000406
1985-10-04,110.074997,-0.007171
1985-10-07,108.199997,-0.017034
...,...,...
2022-03-07,13319.379880,-0.037466
2022-03-08,13267.610350,-0.003887
2022-03-09,13742.200200,0.035771


In [3]:
ndxm = pd.DataFrame(ndx.dailyReturn.resample("M").apply(lambda x: ((x + 1).cumprod() - 1).last("D")))
ndxm.rename(columns={'dailyReturn': 'monthlyReturn'}, inplace=True)
ndxm['downMonths'] = 0

In [4]:
counter = 0
def incrDownMonths(x):
    global counter
    if x['monthlyReturn'] >= 0:
        counter = 0
        return counter
    else:
        counter += 1
        return counter
        

In [5]:
ndxm['downMonths'] = ndxm.apply(incrDownMonths, axis=1)

In [6]:
ndxm

,monthlyReturn,downMonths
date,,
1985-10-31,0.029784,0
1985-11-30,0.091618,0
1985-12-31,0.049461,0
1986-01-31,0.004762,0
1986-02-28,0.056423,0
...,...,...
2021-11-30,0.018009,0
2021-12-31,0.011413,0
2022-01-31,-0.085173,1


In [7]:
ndxm.loc[ndxm.downMonths >=4,'downMonths']

date
1990-10-31    4
2000-12-31    4
2002-07-31    4
2002-08-31    5
2002-09-30    6
2005-04-30    4
2006-07-31    4
2008-02-29    4
2018-12-31    4
Name: downMonths, dtype: int64